for checking the genetic correlation between height and BMI with different BMI traits
for ext. data fig. 1e

read in libraries

In [ ]:
library(openxlsx)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(scales)
library(gsubfn)

read in tables
all downloaded from the neale lab

In [ ]:
hr_raw_small<-read.xlsx("ukb31063_h2_topline.02Oct2019.xlsx")
corr_raw_bmi<-read.csv2("data-2021-12-07_BMI.csv", header=T,sep=",")
corr_raw_ht<-read.csv2("data-2021-12-07_STANDING_HEIGHT.csv", header=T,sep=",")
corr_raw<-read.xlsx("ukbb_geno_correlation_sig.r2.xlsx")
corr_bmi<-subset(corr_raw, corr_raw$p2=='21001_irnt')

Subset heritability (Hr) table to get list of traits with comparable heritability- no correlation taken into account
https://nealelab.github.io/UKBB_ldsc/h2_browser.html
explanation of variables in hr table

In [ ]:
corr_raw_bmi$Description<-strapplyc(corr_raw_bmi$Phenotype.1, "'>(.*)</a>", simplify = TRUE)
corr_raw_ht$Description<-strapplyc(corr_raw_ht$Phenotype.1, "'>(.*)</a>", simplify = TRUE)

hr_raw_small<-hr_raw_small[hr_raw_small$sex=='both_sexes',]
#hr_raw_small<-hr_raw_small[hr_raw_small$notes!='isSexBias;',]

#reformat so can be selected for in same way as hr_raw_small as the phenotypes are listed differently
hr_raw_small<-hr_raw_small[hr_raw_small$description %in% corr_raw_bmi$Description,]
hr_raw_small<-hr_raw_small[hr_raw_small$description %in% corr_raw_ht$Description,]


hr_tbl_binary<-subset(hr_raw_small, hr_raw_small$isBinary=="TRUE" & hr_raw_small$h2_liability>=0.15)
hr_tbl_cont<-subset(hr_raw_small, (hr_raw_small$isBinary!="TRUE" & hr_raw_small$variable_type !='ordinal' & hr_raw_small$h2_observed>=0.15))

hr_tbl_cont<-subset(hr_tbl_cont, (hr_tbl_cont$confidence=="high" | hr_tbl_cont$confidence=="medium"))
hr_tbl_binary<-subset(hr_tbl_binary, (hr_tbl_binary$confidence=="high" | hr_tbl_binary$confidence=="medium"))

hr_tbl<-rbind(hr_tbl_cont,hr_tbl_binary)

corr_bmi<-corr_raw_bmi[corr_raw_bmi$Description %in% hr_tbl$description,]

after trait list of traits is found, need to use table from the version 3 of the Neale lab's datasets
https://docs.google.com/spreadsheets/d/1kvPoupSzsSFBNSztMzl04xMoSC3Kcx3CrjVf4yBmESU/edit#gid=227859291
file downloaded  to xlsx file as UKBB_GWAS_Imputed_v3-File_Manifest_Release_20180731.xlsx

#get the list of  files that need to be downloaded and run in pascal

In [ ]:
manifest<-read.xlsx('UKBB_GWAS_Imputed_v3-File_Manifest_Release_20180731.xlsx', sheet = 'Manifest 201807',colNames = T)
man_subset<-subset(manifest, manifest$Sex=='both_sexes')
man_subset<-subset(man_subset, (man_subset$Phenotype.Code %in% hr_tbl$phenotype))
wget_ls<-as.data.frame(man_subset$wget.command)
wget_ss<- merge(wget_ls, manifest, by.x="man_subset$wget.command", by.y="wget.command", all.x=T, all.y=F)
#get type of data for each trait that will be downloaded and used in network overlap
ss_type<-hr_tbl[hr_tbl$description %in% wget_ss$Phenotype.Description,]
write.table(wget_ss, "neale_imputed_wget_cmds.sh",sep="\b", col.names=F, row.names=F, quote=F)

pre-steps

In [ ]:
%%bash
#download all the raw files- creates .tsv.bgz files
bash neale_imputed_wget_cmds.sh
#download neale variants catalog
wget https://broad-ukb-sumstats-us-east-1.s3.amazonaws.com/round2/annotations/variants.tsv.bgz -O neale_variants.tsv.bgz
#unzip the neale_variants file and raw GWAS results
gunzip -c -q "$file" >> neale_gunzip/"${file%.tsv.bgz}".tsv
#convert to RSID file for running in pascal
#AssignRSID.py in below cell
python /home/bsleger/AssignRSID.py "$file"

AssignRSID.py

In [ ]:
%%python
import pandas as pd
import sys

outfile = sys.argv[1]
outfile = "neale_rsid/" + outfile[0:len(outfile)-31]+"_RSID.txt"
reads = pd.read_csv(sys.argv[1], sep="\t", error_bad_lines=False,low_memory=False)

mapping_df = pd.read_csv("/home/bsleger/bsl/bmi_net/neale_variants.tsv", sep="\t", low_memory=False)

mapping = dict(zip(mapping_df['variant'].tolist(),mapping_df['rsid'].tolist()))
reads['rsid']=reads['variant'].map(mapping)
reads_rmNAN=reads.dropna(subset=['rsid'])
reads_rmNAN.head()
reads_rmNAN= reads_rmNAN[['rsid','pval']]
reads_rmNAN.to_csv(outfile, sep = "\t", index=False)
exit()

In [ ]:
%%bash
#run pascal on RSID file
./Pascal --pval="$file"
#run netcoloc on pascal file using rat BMI seed genes previous calculated
#netcoloc_runscript.py in below cell
bash netcoloc_runscript.py "$file"

netcoloc_runscript.py

In [ ]:
%%python
#system arguments
#------------------ [1]
import os
import datetime
import sys
current_time = datetime.datetime.now() 
print(current_time)
mo_str= str(current_time.month)
if (len(mo_str)==1):
    mo_str='0'+ mo_str
day_str= str(current_time.day)
if (len(day_str)==1):
    day_str='0'+ day_str
date= str(current_time.year) + mo_str + day_str

#define path
#DATADIR= ""
trait= 'rat_BMI'
out_dir= '~' + date
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
human_file_path = sys.argv[1]
human_raw_file = human_file_path
rat_raw_file ='ratBMI_seed_relaxed.txt'
out_network = human_file_path[0:(len(human_file_path)-51)] + '_Neale_sampled_NetColocTo_'+trait + "_relaxed_unsampled"
overlap_label= human_file_path[0:(len(human_file_path)-51)] + '\t'+trait
#change to 0, 51 before restarting the jobs
out_network_file = out_dir + "/" + out_network + '.txt'
out_overlap_file = out_dir + '/' + 'network_overlap.txt'
out_z_random_file= out_dir + '/' + 'sampling_z_random_'+out_network+'.txt'
out_z_file= out_dir+ '/' + 'sampling_z_'+out_network+'.txt'
out_z_final_file= out_dir + '/'+'final_z'+out_network+'.txt'
#------------------ [2]
# load required packages (netcoloc)
import pandas as pd
from netcoloc import netprop_zscore, netprop
import ndex2
import matplotlib.pyplot as plt
from tabulate import tabulate

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random

from IPython.display import display

import getpass
import ndex2

# latex rendering of text in graphs
import matplotlib as mpl
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

sns.set(font_scale=1.4)
sns.set_style('white')
sns.set_style("ticks", {"xtick.major.size": 15, "ytick.major.size": 15})
plt.rcParams['svg.fonttype'] = 'none'

import sys
sys.path.append('../netcoloc/')

from netcoloc import netprop_zscore
from netcoloc import netprop
from netcoloc import network_colocalization

import imp
imp.reload(netprop_zscore)
imp.reload(netprop)
imp.reload(network_colocalization)
# set random seed to enable reproducibility between runs
import random
np.random.seed(1)
#------------------ [3]
#read in data files (netcoloc)
hm_df = pd.read_csv(human_raw_file, sep="\t")
rn_df = pd.read_csv(rat_raw_file,sep="\t")
rn_df.columns = ['gene_symbol']
rn_df.index=rn_df['gene_symbol']
rn_genes = rn_df.index.tolist()
#------------------ [4]
#read in the interactome
#interactome_uuid='4de852d9-9908-11e9-bcaf-0ac135e8bacf' # for PCNet
interactome_uuid='880c7d8c-f5ad-11ec-ac45-0ac135e8bacf' #for rat STRING
ndex_server='public.ndexbio.org'
ndex_user= NA #insert user as string
ndex_password= NA #insert pass as string
G_PC = ndex2.create_nice_cx_from_server(
            ndex_server, 
            username=ndex_user, 
            password=ndex_password, 
            uuid=interactome_uuid
        ).to_networkx()
nodes = list(G_PC.nodes)
pc_nodes = list(G_PC.nodes)
#------------------ [5]
# pre calculate mats used for netprop... this step takes a few minutes, more for denser interactomes
w_prime = netprop.get_normalized_adjacency_matrix(G_PC, conserve_heat=True,weighted=False)
w_double_prime = netprop.get_individual_heats_matrix(w_prime, alpha=.5)
#------------------ [6]
rn_genes = list(np.intersect1d(rn_genes,pc_nodes))
#------------------ [7]
#format human data file - subset to significant gene associations (0.05 corrected by bonferroni)
#df[df.Length > 7] df.column.isin(values)
alpha= 0.05
m=22136
N=500
alpha_corrected = alpha/m

print(len(hm_df))
hm_df_subset=hm_df[hm_df.gene_symbol.isin(pc_nodes)]
hm_df_subset=hm_df[hm_df.pvalue<=(alpha_corrected)]
print(len(hm_df_subset))
hm_genes=hm_df_subset.gene_symbol.tolist()
print(len(hm_genes))
#hm_df_subset = hm_df_subset.sort_values('pvalue',ascending = True).head(500)
#------------------ [8]
indiv_heats=w_double_prime
#------------------ [9]
# -*- coding: utf-8 -*-

"""Functions for getting z-scores from network propagation.
"""

# External library imports
import os
import warnings
from tqdm.auto import tqdm
import ndex2
# Internal module convenience imports
from netcoloc.netcoloc_utils import *
from netcoloc.netprop import *
import random as rn


def netprop_zscore(seed_gene_file, seed_gene_file_delimiter=None, num_reps=10, alpha=0.5, minimum_bin_size=10,
                   interactome_file=None, interactome_uuid='f93f402c-86d4-11e7-a10d-0ac135e8bacf',
                   ndex_server='public.ndexbio.org', ndex_user=None, ndex_password=None, out_name='out',
                   save_z_scores=True, save_final_heat=True, save_random_final_heats=True, verbose=True):
    """
    Performs network heat propagation on the given interactome with the given
    seed genes, then returns the z-scores of the final heat values of each node
    in the interactome.

    The z-scores are calculated based on a null model, which is built by running
    the network propagation multiple times using randomly selected seed genes
    with similar degree distributions to the original seed gene set.

    This method returns a tuple containing the following:

    * :py:class:`pandas.Series` containing z-scores for each gene. Gene names comprise the index column
    * :py:class:`numpy.ndarray` containing square matrix where each row contains the final heat scores
      for each gene from a network propagation from random seed genes

    :param seed_gene_file: Location of file containing a delimited list of
            seed genes
    :type seed_gene_file: str
    :param seed_gene_file_delimiter: Delimiter used to separate genes in seed
                                     gene file. Default any whitespace
    :type seed_gene_file_delimiter: str
    :param num_reps: Number of times the network propagation algorithm should
            be run using random seed genes in order to build the null model
    :type num_reps: int
    :param alpha: Number between 0 and 1. Denotes the importance of the
            propagation step in the network propagation, as opposed to the step
            where heat is added to seed genes only. Recommended to be 0.5 or
            greater
    :type alpha: float
    :param minimum_bin_size: minimum number of genes that should be in
            each degree matching bin.
    :type minimum_bin_size: int
    :param interactome_file: Location of file containing the interactome in
            NetworkX gpickle format. Either the interactome_file argument or the
            interactome_uuid argument must be defined.
    :type interactome_file: str
    :param interactome_uuid: UUID of the interactome on NDEx. Either the
            interactome_file argument or the interactome_uuid argument must be
            defined. (Default: The UUID of PCNet, the Parsimonious Composite
            Network: f93f402c-86d4-11e7-a10d-0ac135e8bacf)
    :type interactome_uuid: str
    :param ndex_server: NDEx server on which the interactome is stored.
            Only needs to be defined if interactome_uuid is defined
    :type ndex_server: str
    :param ndex_user: NDEx user that the interactome belongs to. Only
            needs to be defined if interactome_uuid is defined, and the
            interactome is private
    :type ndex_user: str
    :param ndex_password: password of the NDEx user's account. Only needs
            to be defined if interactome_uuid is defined, and the interactome is
            private
    :type ndex_password: str
    :param out_name: Prefix for saving output files
    :type out_name: str
    :param save_z_scores:
    :param save_final_heat: If ``True``, then the raw network
            propagation heat scores for the original seed gene set will be saved
            in the form of a tsv file in the current directory
    :type save_final_heat: bool
    :param save_random_final_heats: If ``True``, then the raw
            network propagation heat scores for every repetition of the
            algorithm using random seed genes will be saved in the form of a tsv
            file in the current directory. (Beware: This can be a large file if
            num_reps is large.)
    :type save_random_final_heats: bool
    :param verbose: If ``True``, then progress information will
            be logged. Otherwise, nothing will be printed
    :return: (:py:class:`pandas.Series`, :py:class:`numpy.ndarray`)
    :rtype: tuple
    :raises TypeError: If neither interactome_file or interactome_uuid is provided or if
                       **num_reps** is not an ``int``
    """
    # Process arguments

    # seed_gene_file
    seed_gene_file = os.path.abspath(seed_gene_file)
    # num_reps
    try:
        num_reps = int(num_reps)
    except TypeError:
        raise TypeError("The num_reps argument should be an integer")
    # int_file and int_uuid
    if interactome_file is None and interactome_uuid is None:
        raise TypeError("Either interactome_file or interactome_uuid argument must be provided")

    # Load interactome
    if verbose:
        print('Loading interactome')
    if interactome_file is not None:
        interactome_file = os.path.abspath(interactome_file)
        # interactome = nx.Graph()
        interactome = nx.read_gpickle(interactome_file)
    else:
        interactome = ndex2.create_nice_cx_from_server(
            ndex_server,
            username=ndex_user,
            password=ndex_password,
            uuid=interactome_uuid
        ).to_networkx()
    if 'None' in interactome.nodes():
        interactome.remove_node('None')
    nodes = list(interactome.nodes)

    # Log interactome num nodes and edges for diagnostic purposes
    if verbose:
        print('Number of nodes: ' + str(len(interactome.nodes)))
        print('Number of edges: ' + str(len(interactome.edges)))

    # Load seed genes
    seed_file = open(seed_gene_file, 'r')
    seed_genes = list(np.intersect1d(nodes, seed_file.read().split(seed_gene_file_delimiter)))
    if verbose:
        print('\nNumber of seed genes in interactome: ' + str(len(seed_genes)))

    # Calculate individual_heats_matrix from interactome
    if verbose:
        print('\nCalculating w_prime')
    w_prime = get_normalized_adjacency_matrix(interactome, conserve_heat=True)
    if verbose:
        print('\nCalculating individual_heats_matrix')
    individual_heats_matrix = get_individual_heats_matrix(w_prime, alpha)

    # Calculate the z-score
    if verbose:
        print('\nCalculating z-scores: ' + seed_gene_file)
    z_scores, final_heat, random_final_heats = calculate_heat_zscores(
        individual_heats_matrix,
        nodes,
        dict(interactome.degree),
        seed_genes,
        num_reps=num_reps,
        minimum_bin_size=minimum_bin_size)

    # Save z-score results
    z_scores.name = 'z-scores'
    if save_z_scores:
        z_scores.to_csv(out_z_file, sep='\t')

    # If save_final_heat is true, save out the final heat vector
    if save_final_heat:
        final_heat_df = pd.DataFrame(final_heat, columns=['z-scores'])
        final_heat_df.to_csv(out_z_final_file, sep='\t')

    # If save_random_final_heats is true, save out the vector of randoms (this can be a large file)
    if save_random_final_heats:
        random_final_heats_df = pd.DataFrame(
            random_final_heats.T,
            index=nodes,
            columns=range(1, random_final_heats.shape[0] + 1)
        )
        random_final_heats_df.to_csv(out_z_random_file, sep='\t')

    return z_scores, random_final_heats


def calculate_heat_zscores(individual_heats_matrix, nodes, degrees, seed_genes, num_reps=1000,
                           minimum_bin_size=10, random_seed=1):
    """
    Helper function to perform network heat propagation using the given
    individual heats matrix with the given seed genes and return the z-scores of
    the final heat values of each node.

    The z-scores are calculated based on a null model, which is built by running
    the network propagation multiple times using randomly selected seed genes
    with similar degree distributions to the original seed gene set.

    The returned tuple contains the following:

    * :py:class:`pandas.Series` containing z-scores for each gene. Gene names comprise the index column
    * :py:class:`pandas.Series` containing the final heat scores for each gene. Gene names comprise the index column,
    * :py:class:`numpy.ndarray` containing square matrix in which each row contains the final heat scores for each gene
    from a network propagation from random seed genes)

    :param individual_heats_matrix: output of the
            netprop.get_individual_heats_matrix. A square matrix containing the
            final heat contributions of each gene
    :type individual_heats_matrix: :py:class:`numpy.ndarray`
    :param nodes: nodes, in the order in which they were supplied to
            the :py:func:`~netcoloc.netprop.get_normalized_adjacency_matrix` method
            which returns the precursor to the individual_heats_matrix
    :type nodes: list
    :param degrees: Mapping of node names to node degrees
    :type degrees: dict
    :param seed_genes: list of genes to use for network propagation. The
            results of this network propagation will be compared to a set of
            random results in order to obtain z-scores
    :type seed_genes: list
    :param num_reps: Number of times the network propagation algorithm should
            be run using random seed genes in order to build the null model
    :type num_reps: int
    :param minimum_bin_size: minimum number of genes that should be in
            each degree matching bin
    :type minimum_bin_size: int
    :param random_seed:
    :return: (:py:class:`pandas.Series`, :py:class:`pandas.Series`, :py:class:`numpy.ndarray`)
    :rtype: tuple
    """
    # set random seed for reproducibility
    np.random.seed(random_seed)

    # Calculate network propagation results given gene set
    seed_genes = list(np.intersect1d(nodes, seed_genes))
    final_heat = network_propagation(individual_heats_matrix, nodes, seed_genes)

    # Initialize empty matrix for results of random network propagations
    random_final_heats = np.zeros([num_reps, len(final_heat)])

    # Create bins containing genes of similar degree
    bins, actual_degree_to_bin_index = get_degree_binning(degrees, minimum_bin_size)

    # Perform network propagation many times with random seed genes
    for repetition in tqdm(range(num_reps)):
        # Create list of random, degree-matched seed genes
        random_seed_genes = []
        for gene in seed_genes:
            # Find genes with similar degrees to focal gene degree
            degree = degrees[gene]
            genes_of_similar_degree = bins[actual_degree_to_bin_index[degree]]
            # Shuffle the genes in the bin
            np.random.shuffle(genes_of_similar_degree)

            # Add genes to list that haven't already been added
            index = 0
            while genes_of_similar_degree[index] in random_seed_genes:
                index += 1
            random_seed_genes.append(genes_of_similar_degree[index])

        # Perform network propagation with random seed genes
        random_final_heat = network_propagation(individual_heats_matrix, nodes, random_seed_genes)
        # Set seeds to NaN so they don't bias results
        random_final_heat.loc[random_seed_genes] = np.nan
        # Add results to random_final_heats matrix
        random_final_heats[repetition] = random_final_heat

    # Calculate z-scores
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        z_scores = (np.log(final_heat) - np.nanmean(np.log(random_final_heats),
                                                    axis=0)) / np.nanstd(np.log(random_final_heats), axis=0)

    random_final_heats_df = pd.DataFrame(
        random_final_heats.T,
	index=nodes,
	columns=range(1, random_final_heats.shape[0] + 1))
    random_final_heats_df.to_csv(out_z_random_file, sep='\t')        
    final_heat_df = pd.DataFrame(final_heat, columns=['z-scores'])
    final_heat_df.to_csv(out_z_final_file, sep='\t')
    z_scores.to_csv(out_z_file, sep='\t')    
    return z_scores, final_heat, random_final_heats

#change to 500, 100, 1000 before running job
def calculate_heat_zscores_with_sampling(data, nodes, individual_heats, G_PC, trait="BMI", max_genes=500,
                                         num_samples=100,
                                         nominal_sig=0.05, num_reps=1000, out_path="", minimum_bin_size=10):
    """
    Takes a set of summary statistics and a molecular interaction and performs sampling of the significant genes.
    For each sample a random selection of seed genes is chosen, weighted by the p-value of each gene in the summary
    statistics. Network propagation with zscore calculation is performed for each sample to generate a distribution
    of z-scores for each gene in the seed_gene set.
    :param data: Gene level summary statistics
    :param nodes: list of nodes in the interaction network
    :param individual_heats: Heat matrix calculated by `netprop_zscore.get_individual_heats_matrix())
    :param G_PC: molecular interaction network
    :param trait: name of trait being investigated
    :param max_genes: Maximum number of seed genes to include in each sample (default=500, maximum=500)
    :param num_samples: Number of times to perform sampling (default=100)
    :param nominal_sig: Significance cutoff for keeping genes in data (Note: this value will be Bonferroni corrected)
    :param num_reps: Number of repetitions of randomization for generating null distribution for z_scores
    :param out_path: prefix for saving results of sampling
    :param minimum_bin_size: minimum number of genes that should be in
            each degree matching bin
    :type minimum_bin_size: int
    :return:
    """
    assert max_genes <= 500, "NetColoc is only valid for sets of 500 or less genes so maximum number of genes for " \
                             "sampling must be <= 500"
    outfile = out_dir + '/'+ out_network + "sampling_" + str(max_genes) + "_" + str(num_samples) + ".tsv"
    
    data = data.loc[data.gene_symbol.isin(nodes)]  # subset to genes in interaction network
    all_seeds = data.loc[data.pvalue <= nominal_sig / len(data)]  # Bonferroni correction
    all_seeds = all_seeds.assign(log10p=-1 * np.log10(all_seeds.pvalue))  # get -log10p for weighted sampling
    sampling_results = []
    for i in range(num_samples):
        # perform propagation for sample
        sample_seeds = random.choices(population=all_seeds.gene_symbol.values, weights=all_seeds.log10p.values, k=max_genes)
        sample_results = calculate_heat_zscores(individual_heats, nodes=list(G_PC.nodes), degrees=dict(G_PC.degree),
                                                seed_genes=sample_seeds, num_reps=num_reps,
                                                minimum_bin_size=minimum_bin_size, random_seed=i)[0]
        sample_z = pd.DataFrame(sample_results, columns=["z" + str(i)])
        # save running results of sampling
        if i == 0:
            sample_z.to_csv(outfile, sep="\t")
        else:
            existing = pd.read_csv(outfile, sep="\t", index_col=0)
            existing = existing.join(sample_z)
            existing.to_csv(outfile, sep="\t")
        sampling_results.append(sample_z)

    return pd.concat(sampling_results, axis=1)


def get_consensus_z_scores(sampled_results, percentile=.75):
    """
    returns the consensus z score for each gene across all samples
    :param sampled_results: output of netprop_zscore.calculate_heat_zscores_with_sampling
    :type sampled_results: str (file path) or pandas.DataFrame
    :param percentile: Percentile cut off for determining consensus score (default=0.75)
    :type percentile: float
    :return: Consensus z-scores for all genes based on sampling
    """
    if type(sampled_results) == str:
        results = pd.read_csv(sampled_results, sep="\t", index_col=0)
    else:
        results = sampled_results
    consensus_z = pd.DataFrame({'z': results.quantile(q=percentile, axis=1)})
    return consensus_z
#------------------ [10]
if (len(hm_genes)>500):
    sampled_data = calculate_heat_zscores_with_sampling(hm_df_subset, nodes, indiv_heats, G_PC, trait="BMI")
    z_hm = get_consensus_z_scores(sampled_data, percentile=0.75)
else:
    z_hm, Fnew_hm, Fnew_rand_hm = calculate_heat_zscores(w_double_prime, pc_nodes, dict(G_PC.degree), hm_genes)
#100, 1000
    z_hm = pd.DataFrame({'z':z_hm})
#------------------ [11]
print('\nCalculating human gene z-scores: ')
z_rn, Fnew_rn, Fnew_rand_rn = calculate_heat_zscores(w_double_prime, pc_nodes, dict(G_PC.degree), rn_genes, num_reps=10,minimum_bin_size=10)
#100, 1000
z_rn = pd.DataFrame({'z':z_rn})
#------------------ [12]
from scipy.stats import hypergeom
from scipy.stats import norm

# ------ customize this section based on your gene sets and how they should be labeled -------
z_dict = {'hm_genes':z_hm,'rn_genes':z_rn}

seed_dict = {'hm_genes':hm_genes,'rn_genes':rn_genes}
# --------------------------------------------------------------------------------------------
# save the num overlap and overlap p-val in dataframes
focal_diseases = ['hm_genes','rn_genes']
network_num_overlap = pd.DataFrame(np.zeros((len(focal_diseases),len(focal_diseases))),index=focal_diseases)
network_num_overlap.columns = focal_diseases

network_obs_exp = pd.DataFrame(np.zeros((len(focal_diseases),len(focal_diseases))),index=focal_diseases)
network_obs_exp.columns = focal_diseases

network_pval_overlap = pd.DataFrame(np.ones((len(focal_diseases),len(focal_diseases))),index=focal_diseases)
network_pval_overlap.columns = focal_diseases

network_exp_mean_overlap = pd.DataFrame(np.ones((len(focal_diseases),len(focal_diseases))),index=focal_diseases)
network_exp_mean_overlap.columns = focal_diseases

network_exp_std_overlap = pd.DataFrame(np.ones((len(focal_diseases),len(focal_diseases))),index=focal_diseases)
network_exp_std_overlap.columns = focal_diseases

zthresh=3
for i in np.arange(len(focal_diseases)-1):
    for j in np.arange(1+i,len(focal_diseases)):
        d1=focal_diseases[i]
        d2=focal_diseases[j]
        
        seed1 = seed_dict[d1]
        seed2 = seed_dict[d2]
        
        z1=z_dict[d1]
        z1_noseeds = z1.drop(list(np.intersect1d(seed1+seed2,z1.index.tolist())))
        z2=z_dict[d2]
        z2_noseeds = z2.drop(list(np.intersect1d(seed1+seed2,z2.index.tolist())))
        # replace hypergeometric with permutation empirical p
#         z_d1d2_size,high_z_rand=network_colocalization.calculate_expected_overlap(d1,d2,z1_noseeds,z2_noseeds,
#                                                            plot=False,numreps=1000,zthresh=zthresh)

        z_d1d2_size,high_z_rand=network_colocalization.calculate_expected_overlap(z_scores_1=z1['z'],z_scores_2=z2['z'],z1_threshold= 1.5,z2_threshold= 1.5,
                                                           z_score_threshold=zthresh,num_reps=1000, plot=False)        
        ztemp = (z_d1d2_size-np.mean(high_z_rand))/np.std(high_z_rand)
        ptemp = norm.sf(ztemp)
        print(d1+' + '+d2)
        print('size of network intersection = '+str(z_d1d2_size))
        print(z_d1d2_size)
        print(np.mean(high_z_rand))
        obs_exp_temp = float(z_d1d2_size)/np.mean(high_z_rand)
        print('observed size/ expected size = ' + str(obs_exp_temp))
        print('p = '+ str(ptemp))
        
        
        network_num_overlap.loc[d1][d2]=z_d1d2_size
        network_num_overlap.loc[d2][d1]=z_d1d2_size

        network_pval_overlap.loc[d1][d2]=ptemp
        network_pval_overlap.loc[d2][d1]=ptemp
        
        network_obs_exp.loc[d1][d2]=obs_exp_temp
        network_obs_exp.loc[d2][d1]=obs_exp_temp
        
        network_exp_mean_overlap.loc[d1][d2]=np.mean(high_z_rand)
        network_exp_mean_overlap.loc[d2][d1]=np.mean(high_z_rand)
        
        network_exp_std_overlap.loc[d1][d2]=np.std(high_z_rand)
        network_exp_std_overlap.loc[d2][d1]=np.std(high_z_rand)
        overlap_values = [z_d1d2_size, high_z_rand, obs_exp_temp, ptemp]
#------------------ [13]
# select genes in network intersection, make a subgraph
d1='hm_genes'
d2='rn_genes'
z1=z_dict[d1]
z2=z_dict[d2]

G_overlap = network_colocalization.calculate_network_overlap_subgraph(G_PC,z1['z'],z2['z'],z_score_threshold=3)
#print(len(G_overlap.nodes()))
#print(len(G_overlap.edges()))
#------------------ [14]
# compile dataframe of metadata for overlapping nodes
node_df = pd.DataFrame(index=list(G_overlap.nodes))
node_df[d1+'_seeds']= 0
node_df[d2+'_seeds']= 0
node_df[d1+'_seeds'].loc[list(np.intersect1d(seed_dict[d1],node_df.index.tolist()))]=1
node_df[d2+'_seeds'].loc[list(np.intersect1d(seed_dict[d2],node_df.index.tolist()))]=1
node_df['z_'+d1]=z1.loc[list(G_overlap.nodes)]['z']
node_df['z_'+d2]=z2.loc[list(G_overlap.nodes)]['z']
node_df['z_both']=node_df['z_'+d1]*node_df['z_'+d2]

node_df = node_df.sort_values('z_both',ascending=False)
#------------------ [15]
mean_highz= np.mean(high_z_rand)
stdev_highz=np.std(high_z_rand)
nhgenes=len(hm_df)
nrgenes=len(rn_df)
overlap_values = [overlap_label ,str(z_d1d2_size), str(mean_highz), str(stdev_highz), str(obs_exp_temp), str(ptemp), str(nhgenes), str(nrgenes)]
#------------------ [16]
#write output files
#out_overlap_file
#out_network_file
#with open(out_overlap_file, 'a+')  # open file in append mode
#f.write('\n' overlap_values)
sourceFile = open(out_overlap_file, 'a+')
joined_list = '\t'.join(overlap_values)
print(joined_list , file = sourceFile)
sourceFile.close()
with open(out_network_file, 'w') as f:
    f.write(tabulate(node_df))
#------------------ [17]
# ----- a number of properties should be customized here ------
#Annotate network
G_overlap_cx = ndex2.create_nice_cx_from_networkx(G_overlap)
G_overlap_cx.set_name(out_network) 
for node_id, node in G_overlap_cx.get_nodes():
    data = node_df.loc[node['n']]
    for row, value in data.items():
        if row == 'hm_seeds' or row == 'rn_seeds':
            data_type = 'boolean'
            if value == 0:
                value = False
            else:
                value = True
        else:
            data_type = 'double'
        G_overlap_cx.set_node_attribute(node_id, row, value, type=data_type)
#Upload to NDEx
SERVER ='ndexbio.org'
USERNAME = ndex_user
PASSWORD = ndex_password
network_uuid = G_overlap_cx.upload_to(SERVER, USERNAME, PASSWORD)
#------------------ [18]

combine the network overlap files for all different genes

In [ ]:
%%bash
cat ~/*/network_overlap.txt > ~/network_overlap.txt

read in network overlap file

In [ ]:
setwd("~/Desktop/")
net_overlap_BMI<-read.table('control trait network overlap/network_overlap.tsv',header=F, sep="\t")
#overlap_values = [overlap_label ,str(z_d1d2_size), str(mean_highz), str(stdev_highz), str(obs_exp_temp), str(ptemp), str(nhgenes), str(nrgenes)]

colnames(net_overlap_BMI)<-c('neale_trait' ,'rat_trait', 'z_size_d1d2','mean_high_z_rand', 'stdev_highz', 'obs_exp_temp', 'ptemp', 'nhgenes', 'nrgenes')
net_overlap_BMI$neale_trait<-sub("*.gwas.imp", "", net_overlap_BMI$neale_trait)

calculate values for plotting functions in net_overlap_BMI

In [ ]:
net_overlap_BMI$yerr_lower = net_overlap_BMI$obs_exp_temp - (net_overlap_BMI$z_size_d1d2/ (net_overlap_BMI$mean_high_z_rand+(1.96*net_overlap_BMI$stdev_highz)))
net_overlap_BMI$yerr_upper = net_overlap_BMI$z_size_d1d2/ (net_overlap_BMI$mean_high_z_rand-(1.96*net_overlap_BMI$stdev_highz)) -net_overlap_BMI$obs_exp_temp

add in meta data (manifest)

In [ ]:
man_subset<-subset(manifest, manifest$Sex=='both_sexes')

net_overlap_BMI<-merge(net_overlap_BMI, man_subset[1:2], by.x='neale_trait', by.y='Phenotype.Code', all.x=T,all.y=F)
net_combined_BMI<-merge(net_overlap_BMI, corr_raw_bmi[c("Description", "rg", "rg.SE")],by.x='Phenotype.Description', by.y="Description", all.x=T,all.y=F) 

hr_tbl_distinct<-distinct(hr_tbl, description, .keep_all=T)
net_combined_BMI<-merge(net_combined_BMI, hr_tbl[c('phenotype', 'h2_observed', 'h2_observed_se', 'h2_liability','h2_liability_se', 'n','variable_type')], by.x='neale_trait', by.y='phenotype', all.x=T, all.y=F)
net_combined_BMI<-subset(net_combined_BMI, (net_combined_BMI$z_size_d1d2!=0))

calculate number of significant human genes per file

In [ ]:
%%bash
bash tabulate_siggenes_totable.py "$file"

tabulate_siggenes_totable.py

In [ ]:
%%python
import os
import sys
import pandas as pd
from tabulate import tabulate
import numpy as np

human_raw_file = sys.argv[1]

out_file = 'human_sig_genes.txt'

pc_nodes=(pd.read_csv('/home/bsleger/bsl/bmi_net/neale_runscripts/pcnet_node_list.tsv',header=None, sep="\t"))
pc_nodes=(pc_nodes.iloc[:, 0]).tolist()

#read in data files (netcoloc)
hm_df = pd.read_csv(human_raw_file, sep="\t")

#format human data file - subset to significant gene associations (0.05 corrected by bonferroni)
#df[df.Length > 7] df.column.isin(values)
alpha= 0.05
m=22136
N=500
alpha_corrected = alpha/m

print(len(hm_df))
hm_df_subset=hm_df[hm_df.gene_symbol.isin(pc_nodes)]
hm_df_subset=hm_df[hm_df.pvalue<=(alpha_corrected)]
print(len(hm_df_subset))
hm_genes=hm_df_subset.index.tolist()
#hm_df_subset = hm_df_subset.sort_values('pvalue',ascending = True).head(500)


with open(out_file, 'a') as f:
        f.write(human_raw_file+ '\t'+ str(len(hm_genes))+ '\n')

read in a table of the number of significant human genes per file first considered as potential seeds, merge with net_combined_BMI

In [ ]:
NHumanGenes=read.table("~/Desktop/control trait network overlap/human_sig_genes.txt", sep="\t", header=F)
colnames(NHumanGenes)=c("file","NHumanGenes")
#NHumanGenes$neale_trait<-grep(NHumanGenes$file, "/home/bsleger/bsl/bmi_net/PASCAL/output/")

getNealeTrait<- function(fstr)
{
  tstr= substr(fstr, 41, nchar(fstr)-51)
  if (grepl('GIANT',fstr))
    tstr= substr(fstr, 41, nchar(fstr)-26)
  if (grepl('gwas', tstr))
    tstr=substr(tstr, 1, nchar(tstr)-9)
  return(tstr)
}
NHumanGenes$neale_trait<-lapply(NHumanGenes$file, getNealeTrait)
net_combined_BMI<-merge(net_combined_BMI, NHumanGenes[c('neale_trait', 'NHumanGenes')], by='neale_trait', all.x=T, all.y = F)
#get distinct network overlap
net_combined_BMI<-distinct(net_combined_BMI, neale_trait, .keep_all=T)

subset net_combined_BMI of heritability of +/- .1 of BMI (.25), N Human Genes implicated from PASCAL>=500

In [ ]:
net_bin<-net_combined_BMI[net_combined_BMI$variable_type=='binary',]
net_bin<-net_bin[net_bin$h2_liability<0.35,]
net_bin<-net_bin[net_bin$h2_liability>0.15,]

net_lin<-net_combined_BMI[net_combined_BMI$variable_type!='binary',]
net_lin<-net_lin[net_lin$h2_liability<0.35,]
net_lin<-net_lin[net_lin$h2_liability>0.15,]
net_combined_BMI<-rbind(net_bin, net_lin, net_combined_BMI[net_combined_BMI$neale_trait=='GIANT_BMI',])
net_combined_BMI<-net_combined_BMI[!is.na(net_combined_BMI$neale_trait),]
net_combined_BMI<-distinct(net_combined_BMI,neale_trait,.keep_all=T)

make table for labeling genetic correlation values

In [ ]:
net_combined_BMI_corr<-net_combined_BMI[!is.na(net_combined_BMI$rg),]
stdev_rg<-sd(net_combined_BMI_corr$rg, na.rm = TRUE)
mean_rg<-mean(as.double(net_combined_BMI_corr$rg), na.rm = TRUE)
net_combined_BMI_corr$rg_zscore<-(as.double(net_combined_BMI_corr$rg)-mean_rg)/stdev_rg
net_combined_BMI_corr$rg_abs<-(abs(as.double(net_combined_BMI_corr$rg)))
net_combined_BMI_corr<-distinct(net_combined_BMI_corr, neale_trait, .keep_all=T)

#write table for network overlap

In [ ]:
write.table(net_combined_BMI, "~/Desktop/network_combined_BMI.txt", sep="\t", quote=F, row.names=F)
write.table(net_combined_BMI_corr, "~/Desktop/network_combined_BMI_corr.txt", sep="\t", quote=F, row.names=F)